In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, log_loss, accuracy_score
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import random

In [2]:
data_dir = '../input/animaldataset/Images/val'

Name = os.listdir(data_dir)
print(Name)
print(len(Name))

In [3]:
N=list(range(len(Name)))
normal_mapping=dict(zip(Name,N)) 
reverse_mapping=dict(zip(N,Name)) 

In [4]:
train_dir ="../input/animaldataset/Images/train"
val_dir = "../input/animaldataset/Images/val"
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(train_dir, 
                                                                    batch_size = 16,
                                                                    image_size = (256,256),
                                                                    shuffle = True, 
                                                                    seed = 505)
validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(val_dir, 
                                                                    batch_size = 16,
                                                                    image_size = (256,256),
                                                                    shuffle = True, 
                                                                    seed = 505)

In [5]:
from glob import glob
img_list = glob("../input/animaldataset/Images/train/*")
print(img_list)

In [6]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)

In [7]:
from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint(filepath = 'best_model.h5', verbose=1, monitor='val_loss',save_best_only=True, mode='auto')  

In [8]:
preprocess_input = tf.keras.applications.resnet.preprocess_input

In [9]:
pretrained_model3 = tf.keras.applications.resnet.ResNet50(input_shape=(256,256,3),include_top=False,weights='imagenet')
pretrained_model3.trainable = True
#Transfer learning 

In [10]:
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
  tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
])

In [11]:
inputs = tf.keras.Input(shape=(256,256,3))
x = data_augmentation(inputs)
x1 = tf.keras.applications.resnet.preprocess_input(x)
x1 = pretrained_model3(x1, training=False)
x3 = tf.keras.layers.Dropout(0.2)(x1)
# x3 = tf.keras.layers.Dense(128, activation='relu')(pretrained_model3.output)
x3 = tf.keras.layers.GlobalAveragePooling2D()(x3)
outputs3 = tf.keras.layers.Dense(90,activation = "sigmoid")(x3)
model = tf.keras.Model(inputs=inputs, outputs=outputs3)

In [12]:
base_learning_rate = 0.00001
model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

In [13]:
history = model.fit(train_dataset,
                    epochs=7,
                    validation_data=validation_dataset, callbacks= [checkpoint])

In [14]:
model.save("model.h5")

In [15]:
image = cv2.imread('../input/animaldataset/Images/test/Img-1.jpg')

In [16]:
def predict (img,model):
    img = cv2.resize(img,dsize=(256,256),interpolation=cv2.INTER_CUBIC)
    img = np.expand_dims(img, axis = 0)
    prediction = model.predict(img)
    cname =np.argmax(prediction)
    animal = Name[cname]    
    return animal
    
normal_mapping['antelope']
    

In [17]:
predict(image,model)